In [1]:
import os
import sys
import tensorflow as tf
import _pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as backend
%matplotlib inline

In [2]:
dataset = pkl.load(open('dataset.pkl', 'rb'))
y = np.load('y_50.npy')

In [3]:
age = 'Age6-9'
age_patients = list(dataset[age].keys())

## Read Data

In [4]:
X = np.stack([dataset[age][p] for p in age_patients], 0)
X = np.stack([X, X, X], -1)
y = np.stack([y for p in age_patients], 0)
del dataset

In [5]:
train_patients = np.random.choice(len(X), int(len(age_patients) * 0.8), replace=False)
test_patients = np.array(list(set(np.arange(len(age_patients))) - set(train_patients)))
X_train, y_train = X[train_patients], y[train_patients]
X_test, y_test = X[test_patients], y[test_patients]
del X
del y

## Params

In [6]:
problem = 'classification'
learning_rate = 1e-3
batch_size = 32
num_epochs = 100
epoch_size = X_train.shape[0] * X_train.shape[1] // batch_size
if problem == 'classification':
    num_classes = len(set(y_train.reshape(-1)))
elif problem == 'regression':
    num_classes = 1
else:
    raise ValueError('Problem type invalid!')

## Batch generator

In [7]:
## First preprocess func: data normalizer
std = np.std(np.concatenate(X_train, 0), (0, -1), keepdims=True)
mean = np.mean(np.concatenate(X_train, 0), (0, -1), keepdims=True)
normalizer_fn = lambda x: (x - mean) / np.clip(std, 1e-12, None)
## Second preprocess funcs: ...
## Data generator
def batch_generator(X, y, batch_size, preprocess_funcs=[]):
    
    while True:
        batch_patients = np.random.choice(
            len(X), batch_size, replace=True)
        X_batch, y_batch = [], []
        for patient in batch_patients:
            idx = np.random.choice(len(X[patient]))
            X_batch.append(X[patient][idx])
            y_batch.append(y[patient][idx])
        X_batch = np.array(X_batch)
        for func in preprocess_funcs:
            X_batch = func(X_batch)
        yield np.array(X_batch), np.array(y_batch)

## Define Model

Reset graph

In [48]:
tf.reset_default_graph()

In [49]:
inp = layers.Input(shape=(111, 129, 3))
label = layers.Input(shape=(), dtype=tf.int32)

Example 1: Custom Network

In [50]:
x = inp
for num_layer in range(5):
    filters = min(256, 2**(num_layer+6))
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(
        filters=filters, kernel_size=(3,3), strides=(2, 2),
        activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
pre_global_pooled = layers.Conv2D(filters=num_classes, kernel_size=(3,3))(x)
global_pooled = layers.GlobalAveragePooling2D()(pre_global_pooled)

Example 2: VGG

In [137]:
from tensorflow.keras.applications import mobilenet_v2

In [138]:
model = mobilenet_v2.MobileNetV2(
    input_shape=(X_train.shape[-3:]), include_top=False, weights='imagenet')

/usr/lib/python3/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [139]:
x = model(mobilenet_v2.preprocess_input(inp))
pre_global_pooled = layers.Conv2D(filters=num_classes, kernel_size=(1, 1))(x)
global_pooled = layers.GlobalAveragePooling2D()(pre_global_pooled)

Define Loss and outputs

In [51]:
if problem == 'classification':
    logits = global_pooled
    preds = layers.Softmax()(logits)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)(
        y_true=tf.one_hot(label, 2), y_pred=preds)
    predicted_label = tf.cast(tf.argmax(preds, -1), tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(label, predicted_label), tf.float32))
elif problem == 'regression':
    preds = tf.squeeze(global_pooled)
    tf.keras.losses.MeanSquaredError()(y_true=label, y_pred=preds)

Define optimizer and training op

In [52]:
trainable_params = tf.trainable_variables()
## Change 'trainable_params' for the case of transfer learning
gradient = tf.gradients(loss, trainable_params)
optimizer = tf.keras.optimizers.Adam(learning_rate)
optimizer_op = optimizer.apply_gradients(zip(gradient, trainable_params))

## Session

In [53]:
sess = tf.Session()
intializer = tf.global_variables_initializer()

## Run training

In [54]:
sess.run(intializer)

In [55]:
a = sess.run(tf.trainable_variables()[0])

In [58]:
b = sess.run(tf.trainable_variables()[0])
np.linalg.norm(a - b)/np.linalg.norm(a)

0.13956156

In [ ]:
backend.set_learning_phase(1)
batch = batch_generator(X_train[:][:, ::50], y_train[:][:, ::50], batch_size, [normalizer_fn])
for epoch in range(100):
    print('Epoch {}'.format(epoch))   
    epoch_training_loss = []
    for iteration in range(epoch_size):
        if iteration % (epoch_size//10) == 0:
            print('\b.', end='\r')
        X_batch, y_batch = next(batch)
        _, batch_loss = sess.run([optimizer_op, loss], {inp: X_batch, label: y_batch})
        epoch_training_loss.append(batch_loss)
    print('\nEpoch training loss {}'.format(np.mean(epoch_training_loss)))

In [46]:
backend.set_learning_phase(0)
X_batch, y_batch = next(batch)
sess.run([accuracy], {inp: X_batch, label: y_batch}), np.mean(y_batch)

([0.625], 0.375)